In [78]:
import requests

from bs4 import BeautifulSoup
import pickle
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize

import pandas as pd
import numpy as np
from selenium import webdriver



In [82]:
# op = webdriver.ChromeOptions()
# op.add_argument('headless')



SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 89
Current browser version is 114.0.5735.198 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
Stacktrace:
0   chromedriver                        0x0000000100b7cb59 chromedriver + 2595673
1   chromedriver                        0x0000000101230b33 chromedriver + 9624371
2   chromedriver                        0x000000010092c773 chromedriver + 169843
3   chromedriver                        0x0000000100950bf6 chromedriver + 318454
4   chromedriver                        0x000000010094cdc4 chromedriver + 302532
5   chromedriver                        0x000000010094a032 chromedriver + 290866
6   chromedriver                        0x000000010097d234 chromedriver + 500276
7   chromedriver                        0x00000001009784f3 chromedriver + 480499
8   chromedriver                        0x0000000100952cd6 chromedriver + 326870
9   chromedriver  

In [77]:
soup = BeautifulSoup(webpage.content, 'lxml')
title = soup.find("span", attrs={"id":'productTitle'})

title_string = title.string.strip()

AttributeError: 'NoneType' object has no attribute 'string'

In [85]:
title_string = "Apple EarPods Headphones with Lightning Connector. Microphone with Built-in Remote to Control Music, Phone Calls, and Volume. Wired Earbuds for iPhone"

In [86]:
with open('../models/count_vec.pkl', 'rb') as f:
    (count_vec, title_tfidf, clf) = pickle.load(f)

In [87]:
# Text feature extraction
sw = set(stopwords.words('english'))
# Remove special characters
title_string_clean = re.sub(r'\s+', ' ', re.sub('[^A-Za-z0-9]', ' ', title_string.strip().lower())).strip()
# Tokenize
title_string_clean = word_tokenize(title_string_clean)
# Remove stop words
title_string_clean = [w for w in title_string_clean if w not in sw]
title_string_clean = " ".join(title_string_clean)
title_string_clean

'apple earpods headphones lightning connector microphone built remote control music phone calls volume wired earbuds iphone'

In [96]:
title_vec = count_vec.transform([title_string_clean])
norm_title_vec = title_tfidf.transform(title_vec)

nbrs = clf.kneighbors(norm_title_vec, 10, return_distance=False)
nbrs

array([[14423, 13443,  4946, 24491, 16734, 23734, 18391,  4795, 27565,
        26625]])

In [89]:
df = pd.read_csv('../data/ebay_data_clean.csv')


In [98]:
print(df.iloc[nbrs[0]]['Pageurl'])

14423    https://www.ebay.com/itm/Apple-EarPods-with-Li...
13443    https://www.ebay.com/itm/2-Pack-Earphones-with...
4946     https://www.ebay.com/itm/Lightning-Headphones-...
24491    https://www.ebay.com/itm/NEW-IN-ORIGINAL-IPHON...
16734    https://www.ebay.com/itm/1More-Triple-Driver-I...
23734    https://www.ebay.com/itm/Genuine-Apple-Ear-pod...
18391    https://www.ebay.com/itm/apple-iphone-earbuds/...
4795     https://www.ebay.com/itm/BestGot-Headphones-Ov...
27565    https://www.ebay.com/p/13017016037?iid=1439499...
26625    https://www.ebay.com/p/13017016037?iid=3931943...
Name: Pageurl, dtype: object


In [51]:
x

'14423    https://www.ebay.com/itm/Apple-EarPods-with-Li...\nName: Pageurl, dtype: object'